In [4]:
import requests
from requests.adapters import HTTPAdapter, Retry
import json
import re
import sys
import pandas as pd

In [5]:
url = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28existence%3A1%29+AND+%28ft_signal_exp%3A*%29%29&size=500"
response = requests.get(url)
session = requests.Session()

In [6]:
def get_next_link(headers):
    if "Link" in headers:
        # Cerca qualunque link rel="next"
        links = headers["Link"].split(",")
        for link in links:
            match = re.search(r'<(.+)>; rel="next"', link)
            if match:
                return match.group(1)
    return None

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers.get("x-total-results")
        yield response, total
        batch_url = get_next_link(response.headers)

def get_kingdom(entry):
  if "Fungi" in entry["organism"]["lineage"]:
     kd = "Fungi"
  elif "Viridiplantae" in entry["organism"]["lineage"]:
    kd = "Viridiplantae"
  elif "Metazoa" in entry["organism"]["lineage"]:
    kd = "Metazoa"
  else:
    kd = "Other"
  return kd

In [7]:
ok_entries = []
for response, total in get_batch(url):
    data = response.json()
    for i in range(len(data['results'])):
      entry = data['results'][i]
      if data['results'][i]['features'][0]['location']['end']['value'] > 13 and data['results'][i]['features'][0]['description'] == '':
        ok_entries.append(entry)

In [31]:
with open("positive.tsv", 'w') as output_file, open("positive.fasta", "w") as fasta_outputfile:
  for entry in ok_entries:
    line = f'''{entry["primaryAccession"]}{'\t'}{entry["organism"]["scientificName"]}{'\t'}{get_kingdom(entry)}{'\t'}{entry["sequence"]["length"]}{'\t'}{entry["features"][0]["location"]["end"]["value"]}'''
    output_file.write(line + '\n')
    header = f""">{entry["primaryAccession"]} {entry["organism"]["scientificName"]}"""
    sequence = entry["sequence"]["value"]
    fasta_outputfile.write(header + "\n" + sequence + "\n")

# Negatives

In [17]:
urlo_neg = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28existence%3A1%29+NOT+%28ft_signal%3A*%29+OR+%28cc_scl_term_exp%3ASL-0091%29+OR+%28cc_scl_term_exp%3ASL-0191%29+OR+%28cc_scl_term_exp%3ASL-0173%29+OR+%28cc_scl_term_exp%3ASL-0209%29+OR+%28cc_scl_term_exp%3ASL-0204%29+OR+%28cc_scl_term_exp%3ASL-0039%29%29&size=500"
response_neg = requests.get(urlo_neg)
session = requests.Session()

In [18]:
neg_entries = []
for response, total in get_batch(urlo_neg):
    data_neg = response.json()

In [20]:
data_neg.keys()

dict_keys(['results'])

In [30]:
for i in range(1000):
  if len(data_neg['results'][i]['features'])>1:
    print(data_neg['results'][i]['features'][1])
  #for key in feature:
    #if key['type']=='Transmembrane':

{'type': 'Chain', 'location': {'start': {'value': 30, 'modifier': 'EXACT'}, 'end': {'value': 215, 'modifier': 'EXACT'}}, 'description': 'CUE domain-containing protein 4, mitochondrial', 'featureId': 'PRO_0000310344'}
{'type': 'Region', 'location': {'start': {'value': 84, 'modifier': 'EXACT'}, 'end': {'value': 108, 'modifier': 'EXACT'}}, 'description': 'Disordered', 'evidences': [{'evidenceCode': 'ECO:0000256', 'source': 'SAM', 'id': 'MobiDB-lite'}]}
{'type': 'Region', 'location': {'start': {'value': 20, 'modifier': 'EXACT'}, 'end': {'value': 71, 'modifier': 'EXACT'}}, 'description': 'Disordered', 'evidences': [{'evidenceCode': 'ECO:0000256', 'source': 'SAM', 'id': 'MobiDB-lite'}]}
{'type': 'Modified residue', 'location': {'start': {'value': 200, 'modifier': 'EXACT'}, 'end': {'value': 200, 'modifier': 'EXACT'}}, 'description': 'Phosphoserine', 'evidences': [{'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '18257517'}]}
{'type': 'Region', 'location': {'start': {'value': 166, 'mo

IndexError: list index out of range